In [1]:
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_excel("D:/Temp_Huilin/auc4.xlsx")

In [3]:
df

,Alg,funcID,Dim,Budget,AUC
0,SEP_MFO_asyncE,1,5,50010,0.835990
1,SEP_MFO_asyncE,1,20,200010,0.255928
2,SEP_MFO_asyncE,2,5,50010,0.414592
3,SEP_MFO_asyncE,2,20,200010,0.000000
4,SEP_MFO_asyncE,3,5,50010,0.152617
...,...,...,...,...,...
91,SEP_MFO_syncE,22,20,200010,0.157859
92,SEP_MFO_syncE,23,5,50010,0.288041
93,SEP_MFO_syncE,23,20,200010,0.232679
94,SEP_MFO_syncE,24,5,50010,0.132629


In [5]:
D = 20
stats.wilcoxon(df[(df['Alg'] == 'SEP_MFO_syncE') & (df['Dim'] == D)]['AUC'].values,
               df[(df['Alg'] == 'SEP_MFO_asyncE') & (df['Dim'] == D)]['AUC'].values, mode = 'approx', zero_method = 'pratt')

WilcoxonResult(statistic=138.0, pvalue=0.8409995729722781)

In [20]:
D = 20
stats.wilcoxon(df[(df['Alg'] == 'UNIOA_MFO') & (df['Dim'] == D)]['AUC'].values,
                                    df[(df['Alg'] == 'SEP_MFO') & (df['Dim'] == D)]['AUC'].values,  
                                         alternative="two-sided", mode = 'approx', 
                                         zero_method = 'pratt')

WilcoxonResult(statistic=6.0, pvalue=4.631037499394086e-05)

In [2]:
# SEP_fix VS UNIOA 
SEP_fix_VS_UNIOA = pd.read_excel("D:/Temp_Huilin/auc_14.xlsx")


# syncE VS asyncE
syncE_VS_asyncE = pd.read_excel("D:/Temp_Huilin/auc_28.xlsx")


# syncG VS asyncG
syncG_VS_asyncG = pd.read_excel("D:/Temp_Huilin/auc_10.xlsx")

In [8]:
def wilcoxon_test(df, output_csa_name, propose):
    column_names = ["algorithm", "dim5", "dim20"]
    Algs = ['BA', 'GOA', 'CSA', 'MFO', 'MBO', 'BOA', 'PSO']
    o = pd.DataFrame(columns = column_names)
    zeros = np.zeros(24)
    if propose == 'SEP_fix_VS_UNIOA':
        o['algorithm'] = Algs
        for col in ["dim5", "dim20"]:
            P = []
            if col == "dim5":
                d = 5
            else:
                d = 20
            for A in Algs:
                unioa_name = 'UNIOA_' + A
                if A == 'BA' or A == 'BOA' or A == 'PSO':
                    sep_name = 'SEP_' + A + '_fix'
                else:
                    sep_name = 'SEP_' + A
                p_value = stats.wilcoxon(df[(df['Alg'] == unioa_name) & (df['Dim'] == d)]['AUC'].values,
                                    df[(df['Alg'] == sep_name) & (df['Dim'] == d)]['AUC'].values,  
                                         alternative="less", mode = 'approx', 
                                         zero_method = 'pratt')[1]
                P.append(p_value)            
            o[col] = P
            
    if propose == 'syncE_VS_asyncE':
        sep_algorithms = []
        unioa_algorithms = []
        for A in Algs:
            sep = 'SEP_' + A + '_syncE_or_asyncE_syncG'
            unioa = 'UNIOA_' + A + '_syncE_or_asyncE_syncG'
            sep_algorithms.append(sep)
            unioa_algorithms.append(unioa)
        
        o['algorithm'] = sep_algorithms + unioa_algorithms
        for col in ["dim5", "dim20"]:
            sep_P = []
            unioa_P = []
            if col == "dim5":
                d = 5
            else:
                d = 20
            for A in Algs:
                if A == 'CSA' or A == 'MFO':
                    sep_ss = 'SEP_' + A + '_syncE'
                    sep_as = 'SEP_' + A + '_asyncE'
                    unioa_ss = 'UNIOA_' + A + '_syncE'
                    unioa_as = 'UNIOA_' + A + '_asyncE'
                else:
                    sep_ss = 'SEP_' + A + '_syncE_syncG'
                    sep_as = 'SEP_' + A + '_asyncE_syncG'
                    unioa_ss = 'UNIOA_' + A + '_syncE_syncG'
                    unioa_as = 'UNIOA_' + A + '_asyncE_syncG'
                    
                sep_p_value = stats.wilcoxon(df[(df['Alg'] == sep_ss) & (df['Dim'] == d)]['AUC'].values,
                                    df[(df['Alg'] == sep_as) & (df['Dim'] == d)]['AUC'].values, mode = 'approx', zero_method = 'pratt')[1]
                sep_P.append(sep_p_value)                
                unioa_p_value = stats.wilcoxon(df[(df['Alg'] == unioa_ss) & (df['Dim'] == d)]['AUC'].values, 
                                    df[(df['Alg'] == unioa_as) & (df['Dim'] == d)]['AUC'].values,  mode = 'approx', zero_method = 'pratt')[1]
                unioa_P.append(unioa_p_value)
    
            o[col] = sep_P + unioa_P
            
    if propose == 'syncG_VS_asyncG':
        o['algorithm'] = ['SEP_BA', 'SEP_GOA', 'SEP_MBO', 'SEP_BOA', 'SEP_PSO']
        for col in ["dim5", "dim20"]:
            P = []
            if col == "dim5":
                d = 5
            else:
                d = 20
            for A in Algs:
                if A == 'CSA' or A == 'MFO':
                    pass
                else:
                    sep_as = 'SEP_' + A + '_asyncE_syncG'
                    sep_aa = 'SEP_' + A + '_asyncE_asyncG'
                    sep_p_value = stats.wilcoxon(df[(df['Alg'] == sep_as) & (df['Dim'] == d)]['AUC'].values ,  
                                        df[(df['Alg'] == sep_aa) & (df['Dim'] == d)]['AUC'].values,  mode = 'approx', zero_method = 'pratt')[1]
                    P.append(sep_p_value)                
            o[col] = P    
    print(o)
    return o.to_csv(output_csa_name + '.csv', index=False)

In [9]:
wilcoxon_test(df = SEP_fix_VS_UNIOA, output_csa_name = 'SEP_fix_VS_UNIOA_d5_d20', propose = 'SEP_fix_VS_UNIOA')

  algorithm      dim5     dim20
0        BA  0.943616  0.672069
1       GOA  0.040213  0.967151
2       CSA  0.970333  0.393020
3       MFO  0.999802  0.999977
4       MBO  0.187886  0.070486
5       BOA  0.431943  0.583635
6       PSO  0.996955  0.696796


In [10]:
wilcoxon_test(df = syncE_VS_asyncE, output_csa_name = 'syncE_VS_asyncE_d5_d20', propose = 'syncE_VS_asyncE')

                          algorithm      dim5     dim20
0      SEP_BA_syncE_or_asyncE_syncG  0.331335  0.228869
1     SEP_GOA_syncE_or_asyncE_syncG  0.626111  0.667365
2     SEP_CSA_syncE_or_asyncE_syncG  0.647568  0.886241
3     SEP_MFO_syncE_or_asyncE_syncG  0.002700  0.017373
4     SEP_MBO_syncE_or_asyncE_syncG  0.317311  0.174692
5     SEP_BOA_syncE_or_asyncE_syncG  0.775097  0.277399
6     SEP_PSO_syncE_or_asyncE_syncG  0.954431  0.194499
7    UNIOA_BA_syncE_or_asyncE_syncG  0.241427  0.596845
8   UNIOA_GOA_syncE_or_asyncE_syncG  0.646558  0.623344
9   UNIOA_CSA_syncE_or_asyncE_syncG  0.909011  0.943028
10  UNIOA_MFO_syncE_or_asyncE_syncG  0.647568  0.383277
11  UNIOA_MBO_syncE_or_asyncE_syncG  0.265157  0.246973
12  UNIOA_BOA_syncE_or_asyncE_syncG  0.097491  0.629275
13  UNIOA_PSO_syncE_or_asyncE_syncG  0.954431  0.465518


In [11]:
wilcoxon_test(df = syncG_VS_asyncG, output_csa_name = 'syncG_VS_asyncG_d5_d20', propose = 'syncG_VS_asyncG')

  algorithm      dim5     dim20
0    SEP_BA  0.224591  0.004638
1   SEP_GOA  0.004220  0.173186
2   SEP_MBO  0.198543  0.074116
3   SEP_BOA  0.819202  0.277399
4   SEP_PSO  0.440453  0.964389


In [ ]:
syncE_VS_asyncE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
  
# create data
x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
SEP_MFO_syncE_syncG = syncE_VS_asyncE[(syncE_VS_asyncE['ID'] == 'SEP_MFO_syncE') & (syncE_VS_asyncE['DIM'] == 20)]['auc'].values
SEP_MFO_asyncE_syncG = syncE_VS_asyncE[(syncE_VS_asyncE['ID'] == 'SEP_MFO_asyncE') & (syncE_VS_asyncE['DIM'] == 20)]['auc'].values

# plot lines
plt.plot(x, SEP_MFO_syncE_syncG, label = "ss")
plt.plot(x, SEP_MFO_asyncE_syncG, label = "as")
plt.legend()
plt.xlabel('24 functions')
plt.ylabel('AUC')
#plt.savefig('CD.png')
plt.show()


In [ ]:
o = pd.DataFrame(columns = ['A', 'B', 'difference', 'abs_difference', 'rank'])
o['A'] = SEP_MBO_asyncE_syncG
o['B'] = SEP_MBO_syncE_syncG

In [ ]:
difference = SEP_MBO_asyncE_syncG - SEP_MBO_syncE_syncG
abs_difference = np.abs(difference)
rank =  stats.rankdata(abs_difference)

In [ ]:
o['difference'] = difference
o['abs_difference'] = abs_difference
o['rank'] = rank

In [ ]:
T_minus = o[(o['difference'] < 0)]['rank'].values
T_plus = o[(o['difference'] >0)]['rank'].values

In [ ]:
sum(T_minus), sum(T_plus)

In [ ]:
A = SEP_MBO_asyncE_syncG
B = SEP_MBO_syncE_syncG

In [ ]:
difference = A-B
T_minus = []
T_plus = []
for diff in difference:
    if diff < 0:
        T_minus.append(diff)
    else:
        T_plus.append(diff)

In [ ]:
len(T_minus), len(T_plus)

In [ ]:
abs_difference = np.abs(A - B)

In [ ]:
rank =  stats.rankdata(abs_difference)

In [ ]:

D = []
for i in difference:
    D.append(float(format(i, '.8f')))

In [ ]:
D

In [ ]:
np.savetxt('myfile.csv', D, delimiter=',')

In [ ]:
rank

In [ ]:
stats.rankdata([5,1,0,10])

In [ ]:
stats.wilcoxon(SEP_MBO_asyncE_syncG, SEP_MBO_syncE_syncG, 
               alternative='two-sided', 
               correction=True,
               mode='approx')

In [ ]:
SEP_MBO_asyncE_syncG.mean()-SEP_MBO_syncE_syncG.mean()

In [ ]:
import numpy as np
np.zeros(24)

In [ ]:
t = []
for i in range(-80, 22, 2):
    e = i/10
    t.append(10**e)

In [ ]:
t